In [1]:
from utils.downloader import *
from utils.dataset_reader import *
from utils.extractors import *
import torch
import torchvision
import torch.nn as nn
import os

2024-05-24 09:44:11.246958: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 09:44:11.941670: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# TODO
- Aggiungere nella pipeline di preparazione dei dati di pytorch la possibilità di fare da augmentation.
- I logs sono salvati in datasets al momento


# Notebook usage
This notebook is used to streamline as much as possible the data setup for the project. We have trained models on data downloaded from the internet 
as well as present in the Torch and timm libraries. 

This notebook will help you set up the data in the correct directory structure.
It will be used to download the data, extract it, and set it up in the correct directory structure.
If you want to work using datasets preloaded in Torch or timm, you can skip this notebook.

Everything that needs to be edited is marked with "@edit". It is procedural, therefore do not skip steps and go one cell at the time. 

At the moment less common cases such as image folder + .mat file or .csv file are not yet implemented in this notebook (csv function exists) or not implemented at all (.mat function doesn't exist at all)

## Download Data from Kaggle

In [ ]:
# manually edit the three following lines, save the dataset wherever you wish, but the "datasets" folder is recommended
# make sure to use the absolute path, not relative
# @edit
dataset_url = "https://www.kaggle.com/datasets/prasanshasatpathy/soil-types"
# @edit
folder_target = "/home/lorenzo/Desktop/ml/intromlproject_su cui lavorare/datasets"
download_dataset_kaggle(dataset_url, folder_target)

# Download dataset from a url

In [ ]:
# @edit
dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz"
folder_target = '"/home/disi/ml/datasets/"'
i = download_dataset(dataset_url, folder_target)
# Manually move it in the datasets folder if you do not need to unzip it, otherwhise extracting it with the following function
# will do it for you
# @edit
extract_tgz(f"{dataset_url}/datasets.tar.gz", folder_target)

## Augment the dataset (optional)

In [ ]:
# @edit
# make sure to add a / in front of the folder name 
dataset_folder = folder_target + "/Soil types"
image_generator =  setup_data_generator(rotation_range=40,
                                        width_shift_range=0.2, 
                                        height_shift_range=0.2,
                                        shear_range=0.2, 
                                        zoom_range=0.2, 
                                        horizontal_flip=True, 
                                        fill_mode='nearest')

create_train_val_test_folders(dataset_folder, train_size=0.7, val_size=0.15, test_size=0.15)
cleanup(dataset_folder)

In [ ]:
directory_path  = dataset_folder + "/train"
train_generator = load_data_from_directory(directory_path, target_size=(300,300), batch_size=32)
n_of_batches = 50

#TODO write how the number of batches infuences the actual amount of images generated

In [ ]:
for i, (images, labels) in enumerate(train_generator):
    if i >= n_of_batches:  # Stop after saving images from 50 batches
        break
    save_augmented_images(images, labels, directory_path, train_generator.class_indices)

## Train-Test-Val-Split
If you augmented the images you are good to go, the dataset is ready to be used in the main function. Otherwhise edit and execute the following block of code.


In [ ]:
# @edit
# make sure to add a / in front of the folder name 
dataset_folder = folder_target + "/jpg"

In [ ]:
create_train_val_test_folders(dataset_folder, train_size=0.7, val_size=0.15, test_size=0.15)

# Datasets from Pytorch

## FGVCAircraft

In [2]:
# @edit
root = "/home/disi/ml/datasets/"
dataset = torchvision.datasets.FGVCAircraft(root=root, download = True)

100%|██████████| 2753340328/2753340328 [00:33<00:00, 81301460.42it/s] 


Extracting /home/disi/ml/datasets/fgvc-aircraft-2013b.tar.gz to /home/disi/ml/datasets/


In [3]:
# @edit
dataset_path = f'{root}/fgvc-aircraft-2013b/data/images'
new_dataset_path = f'{root}'
train_txt_path = f'{root}/fgvc-aircraft-2013b/data/images_family_train.txt'
val_txt_path = f'{root}/fgvc-aircraft-2013b/data/images_family_val.txt'
test_txt_path = f'{root}/fgvc-aircraft-2013b/data/images_family_test.txt'

In [4]:
reorganize_dataset_txt(dataset_path, train_txt_path, new_dataset_path, 'train')
reorganize_dataset_txt(dataset_path, val_txt_path, new_dataset_path, 'val')
reorganize_dataset_txt(dataset_path, test_txt_path, new_dataset_path, 'test')

In [5]:
# @edit
cleanup(f'{root}')

Deleted file: /home/disi/ml/datasets/fgvc-aircraft-2013b.tar.gz
Skipped folder: /home/disi/ml/datasets/val
Skipped folder: /home/disi/ml/datasets/train
Deleted folder: /home/disi/ml/datasets/fgvc-aircraft-2013b
Skipped folder: /home/disi/ml/datasets/test


## Flowers102 / Oxford Flowers

In [8]:
# @edit
root = "/home/disi/ml/datasets/"
dataset = torchvision.datasets.Flowers102(root=root, download = True)


100%|██████████| 344862509/344862509 [00:01<00:00, 260280841.69it/s]


Extracting /home/disi/ml/datasets/flowers-102/102flowers.tgz to /home/disi/ml/datasets/flowers-102


100%|██████████| 502/502 [00:00<00:00, 488071.54it/s]


100%|██████████| 14989/14989 [00:00<00:00, 21089709.04it/s]


In [10]:
# Paths to your original dataset, .mat file, and new dataset location
# @edit
dataset_path = f'{root}/flowers-102/jpg'
mat_path = f'{root}/flowers-102/imagelabels.mat'
new_dataset_path = f'{root}'

In [11]:
# Call the function for each of your train, validation, and test sets
reorganize_dataset_mat(dataset_path, mat_path, new_dataset_path, 'train')
reorganize_dataset_mat(dataset_path, mat_path, new_dataset_path, 'val')
reorganize_dataset_mat(dataset_path, mat_path, new_dataset_path, 'test')

In [12]:
# @edit
cleanup(root)

Skipped folder: /home/disi/ml/datasets/val
Deleted folder: /home/disi/ml/datasets/flowers-102
Skipped folder: /home/disi/ml/datasets/train
Skipped folder: /home/disi/ml/datasets/test


# Last step
Once you reach the train-test-val configuration, run this final function to create a folder named as you wish that will contain train-test-val. This will be our root in the main.py. 

In [15]:
# @edi
location = root
newfolder_name = 'Flowers102'
train_folder = f'{location}/train/'
val_folder = f'{location}/val/'
test_folder = f'{location}/test/'
final_structure(location, newfolder_name, train_folder, val_folder, test_folder)

In [ ]:
import os

In [ ]:
img_root = '/home/lorenzo/Desktop/qui/intromlproject_su cui lavorare/datasets/Flowers102'
path = os.path.dirname(img_root) + "/"

In [ ]:
print(img_root)

In [ ]:
folder = os.path.basename(img_root)
print(path)

In [ ]:

# Define the root path
img_root = '/home/disi/ml/intromlproject/datasets/Aerei'

# Standardize the path
standardized_path = os.path.normpath(img_root)

# Split the path into directory and the last component
path, folder = os.path.split(standardized_path)

# Print the results
print("Path:", path)
print("Folder:", folder)